# Lab 5: Spam Detection

**Deadline**: Monday, Mar 14, 5:00 PM

**Late Penalty**: Any work that is submitted between 0 hour and 24 hours past the deadline will receive a 20% grade deduction. No other late work is accepted. Quercus submission time will be used, not your local computer time. You can submit your labs as many times as you want before the deadline, so please submit often and early.

**TA**: Hossein Yousefi <hossein.yousefi@mail.utoronto.ca>

In this assignment, we will build a recurrent neural network to classify a SMS text message
as "spam" or "not spam". In the process, you will
    
1. Clean and process text data for machine learning.
2. Understand and implement a character-level recurrent neural network.
3. Use torchtext to build recurrent neural network models.
4. Understand batching for a recurrent neural network, and use torchtext to implement RNN batching.

### What to submit

Submit a PDF file containing all your code, outputs, and write-up. You can produce a PDF of your Google Colab file by going to File > Print and then save as PDF. The Colab instructions have more information (.html files are also acceptable).

Do not submit any other files produced by your code.

Include a link to your colab file in your submission.

## Colab Link

Include a link to your Colab file here. If you would like the TA to look at your
Colab file in case your solutions are cut off, **please make sure that your Colab
file is publicly accessible at the time of submission**.

Colab Link:

In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

## Part 1. Data Cleaning [15 pt]

We will be using the "SMS Spam Collection Data Set" available at http://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

There is a link to download the "Data Folder" at the very top of the webpage. Download the zip file, unzip it, and upload the file `SMSSpamCollection` to Colab.    

### Part (a) [2 pt]

Open up the file in Python, and print out one example of a spam SMS, and one example of a non-spam SMS.

What is the label value for a spam message, and what is the label value for a non-spam message?

In [2]:
spam_count = 0
non_spam_count = 0
for line in open('SMSSpamCollection'):
    split_line = line.split()
    if (split_line[0] == "ham" and not non_spam_count):
        non_spam_count += 1
        print(line)
    elif (split_line[0] == "spam" and not spam_count):
        spam_count += 1
        print(line)
    elif (spam_count and non_spam_count):
        break
    

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



### Part (b) [1 pt]

How many spam messages are there in the data set?
How many non-spam messages are there in the data set?


In [56]:
spam_count = 0
non_spam_count = 0
for line in open('SMSSpamCollection'):
    split_line = line.split()
    if (split_line[0] == "ham"):
        non_spam_count += 1
    elif (split_line[0] == "spam"):
        spam_count += 1

print("SPAM = " + str(spam_count) + " NON-SPAM = " + str(non_spam_count))

SPAM = 747 NON-SPAM = 4827


### Part (c) [4 pt]

We will be using the package `torchtext` to load, process, and batch the data.
A tutorial to torchtext is available below. This tutorial uses the same
Sentiment140 data set that we explored during lecture.

https://medium.com/@sonicboom8/sentiment-analysis-torchtext-55fb57b1fab8

Unlike what we did during lecture, we will be building a **character level RNN**.
That is, we will treat each **character** as a token in our sequence,
rather than each **word**.

Identify two advantage and two disadvantage of modelling SMS text
messages as a sequence of characters rather than a sequence of words.

In [25]:
# Advantages: model will be able to be more specific with certain characters that are used in spam messages, especially as some messages add random characters. Also, will be more attentive to grammar and 
# can handle abbreviations to words as many text messages have short forms that aren't actually words 

# Disadvantages: less emphasis on the sentence structure and words used, longer memory required to store the sequence information relationships

### Part (d) [1 pt]

We will be loading our data set using `torchtext.data.TabularDataset`. The
constructor will read directly from the `SMSSpamCollection` file. 

For the data file to be read successfuly, we
need to specify the **fields** (columns) in the file. 
In our case, the dataset has two fields: 

- a text field containing the sms messages,
- a label field which will be converted into a binary label.

Split the dataset into `train`, `valid`, and `test`. Use a 60-20-20 split.
You may find this torchtext API page helpful:
https://torchtext.readthedocs.io/en/latest/data.html#dataset

Hint: There is a `Dataset` method that can perform the random split for you.

In [13]:
import torchtext
import random

text_field = torchtext.legacy.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True)       # to turn each character into an integer index
label_field = torchtext.legacy.data.Field(sequential=False,    # not a sequence
                                   use_vocab=False,     # don't need to track vocabulary
                                   is_target=True,      
                                   batch_first=True,
                                   preprocessing=lambda x: int(x == 'spam')) # convert text to 0 and 1

fields = [('label', label_field), ('sms', text_field)]
dataset = torchtext.legacy.data.TabularDataset("SMSSpamCollection", # name of the file
                                        "tsv",               # fields are separated by a tab
                                        fields)

# Data object allows us to create objects for each text element in the data, with tags "sms" and "label" that can be referenced 
# dataset[0].sms
# dataset[0].label
train, valid, test = dataset.split(split_ratio=[0.6,0.2,0.2], stratified=True, random_state=random.getstate())
print("Training size: " + str(len(train)))
print("Validation size: " + str(len(valid)))
print("Test size: " + str(len(valid)))

Training size: 3343
Validation size: 1115
Test size: 1115


### Part (e) [2 pt]

You saw in part (b) that there are many more non-spam messages than spam messages.
This **imbalance** in our training data will be problematic for training.
We can fix this disparity by duplicating spam messages in the training set,
so that the training set is roughly **balanced**.

Explain why having a balanced training set is helpful for training our neural network.

Note: if you are not sure, try removing the below code and train your mode.

In [15]:
# save the original training examples
old_train_examples = train.examples
# get all the spam messages in `train`
train_spam = []
for item in train.examples:
    if item.label == 1:
        train_spam.append(item)
# duplicate each spam message 6 more times
train.examples = old_train_examples + train_spam * 6

# The balance between spam and non-spam data is important for training so that the model does not become biased towards one type of result in the binary classification.
print("Training size: " + str(len(train)))

Training size: 24847


### Part (f) [1 pt]

We need to build the vocabulary on the training data by running the below code.
This finds all the possible character tokens in the training set.

Explain what the variables `text_field.vocab.stoi` and `text_field.vocab.itos` represent.

In [17]:
# Here we create a vocabulary set for all of the possible characters that can be found from the training set. This is like a dictionary of all the available unique characters from the words
# Uses the tokenize function set in the field to build the vocab
text_field.build_vocab(train)

# stoi - string to index: map the dictionary of characters to a unique integer (index within dict)
text_field.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x0000025A65A17E50>>,
            {'<unk>': 0,
             '<pad>': 1,
             ' ': 2,
             'e': 3,
             'o': 4,
             't': 5,
             'a': 6,
             'r': 7,
             'n': 8,
             'i': 9,
             's': 10,
             'l': 11,
             '0': 12,
             'c': 13,
             'u': 14,
             'd': 15,
             'm': 16,
             'h': 17,
             '.': 18,
             'w': 19,
             'p': 20,
             'y': 21,
             '1': 22,
             '2': 23,
             '8': 24,
             'g': 25,
             'T': 26,
             'f': 27,
             '5': 28,
             'E': 29,
             'b': 30,
             'C': 31,
             'S': 32,
             '7': 33,
             '4': 34,
             'v': 35,
             'O': 36,
             'k': 37,
             '3': 38,
             'x': 39,
             

In [18]:
# itos - index to string: list of unique characters. Indices of respective characters within this list can be obtained from stoi
text_field.vocab.itos

['<unk>',
 '<pad>',
 ' ',
 'e',
 'o',
 't',
 'a',
 'r',
 'n',
 'i',
 's',
 'l',
 '0',
 'c',
 'u',
 'd',
 'm',
 'h',
 '.',
 'w',
 'p',
 'y',
 '1',
 '2',
 '8',
 'g',
 'T',
 'f',
 '5',
 'E',
 'b',
 'C',
 'S',
 '7',
 '4',
 'v',
 'O',
 'k',
 '3',
 'x',
 '6',
 'N',
 'R',
 'A',
 '9',
 '!',
 'P',
 'I',
 'U',
 ',',
 'M',
 'W',
 'L',
 'F',
 '/',
 'D',
 'G',
 'B',
 '£',
 'H',
 'Y',
 '-',
 'X',
 '&',
 '?',
 ':',
 'z',
 'V',
 "'",
 'K',
 '*',
 '+',
 'j',
 'J',
 ')',
 '(',
 'q',
 'Q',
 '>',
 '=',
 '@',
 '"',
 ';',
 'Z',
 '#',
 '$',
 '<',
 '[',
 ']',
 '_',
 '%',
 '\x93',
 '¡',
 'ü',
 '’',
 'ú',
 'Ü',
 '‘',
 '\x92',
 '|',
 '…',
 '\x96',
 'é',
 '\t',
 '\n',
 '\\',
 '~',
 '\x91',
 '\x94',
 '^',
 'É',
 'è',
 '–',
 '—']

### Part (g) [2 pt]

The tokens `<unk>` and `<pad>` were not in our SMS text messages.
What do these two values represent?

In [ ]:
# The <unk> token means that the character is not recognized from within the dataset, it's an unknown token
# The <pad> token is used to make sure that sequences are of the same length when passing them into the model

### Part (h) [2 pt]

Since text sequences are of variable length, `torchtext` provides a `BucketIterator` data loader,
which batches similar length sequences together. The iterator also provides functionalities to
pad sequences automatically.

Take a look at 10 batches in `train_iter`. What is the maximum length of the
input sequence in each batch? How many `<pad>` tokens are used in each of the 10
batches?

In [19]:
train_iter = torchtext.legacy.data.BucketIterator(train,
                                           batch_size=32,
                                           sort_key=lambda x: len(x.sms), # to minimize padding
                                           sort_within_batch=True,        # sort within each batch
                                           repeat=False)                  # repeat the iterator for many epochs

In [53]:
for i, batch in enumerate(train_iter):
    if (i == 10):
        break
    else:
        # Each bucket iterator object batch will return the character indices, as well as the length of the batch
        # print(batch.sms)
        # Look through character indices tensor and store length
        sms_lengths = []
        pad_count = 0
        for sms in batch.sms[0]:
            sms_lengths.append(len(sms))
        max_length = max(sms_lengths)
        print("Batch " + str(i) + ": " + "Max Input Sequence Length is " + str(max_length))
        # Look through length of batch tensor and identify padding based on difference between max length and batch length
        for batch_length in batch.sms[1]:
            if batch_length.numpy() != max_length:
                pad_count += (max_length - batch_length.numpy())
        print("Batch " + str(i) + ": " + "Number of <pad> tokens used in entire batch is " + str(pad_count))

Batch 0: Max Input Sequence Length is 159
Batch 0: Number of <pad> tokens used in entire batch is 0
Batch 1: Max Input Sequence Length is 118
Batch 1: Number of <pad> tokens used in entire batch is 36
Batch 2: Max Input Sequence Length is 79
Batch 2: Number of <pad> tokens used in entire batch is 73
Batch 3: Max Input Sequence Length is 137
Batch 3: Number of <pad> tokens used in entire batch is 23
Batch 4: Max Input Sequence Length is 156
Batch 4: Number of <pad> tokens used in entire batch is 10
Batch 5: Max Input Sequence Length is 133
Batch 5: Number of <pad> tokens used in entire batch is 28
Batch 6: Max Input Sequence Length is 51
Batch 6: Number of <pad> tokens used in entire batch is 40
Batch 7: Max Input Sequence Length is 130
Batch 7: Number of <pad> tokens used in entire batch is 15
Batch 8: Max Input Sequence Length is 158
Batch 8: Number of <pad> tokens used in entire batch is 0
Batch 9: Max Input Sequence Length is 162
Batch 9: Number of <pad> tokens used in entire batch 

## Part 2. Model Building [8 pt]

Build a recurrent neural network model, using an architecture of your choosing. 
Use the one-hot embedding of each character as input to your recurrent network.
Use one or more fully-connected layers to make the prediction based on your
recurrent network output.

Instead of using the RNN output value for the final token, another often used
strategy is to max-pool over the entire output array. That is, instead of calling
something like:

```
out, _ = self.rnn(x)
self.fc(out[:, -1, :])
```

where `self.rnn` is an `nn.RNN`, `nn.GRU`, or `nn.LSTM` module, and `self.fc` is a 
fully-connected 
layer, we use:

```
out, _ = self.rnn(x)
self.fc(torch.max(out, dim=1)[0])
```

This works reasonably in practice. An even better alternative is to concatenate the
max-pooling and average-pooling of the RNN outputs:

```
out, _ = self.rnn(x)
out = torch.cat([torch.max(out, dim=1)[0], 
                 torch.mean(out, dim=1)], dim=1)
self.fc(out)
```

We encourage you to try out all these options. The way you pool the RNN outputs
is one of the "hyperparameters" that you can choose to tune later on.

In [57]:
# You might find this code helpful for obtaining
# PyTorch one-hot vectors.

ident = torch.eye(10)
print(ident[0]) # one-hot vector
print(ident[1]) # one-hot vector
x = torch.tensor([[1, 2], [3, 4]])
print(ident[x]) # one-hot vectors

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]])


In [59]:
class spamGRU(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(spamGRU, self).__init__()
        self.name = "RNN"
        # Generate one hot encodings of input
        self.ident = torch.eye(vocab_size)
        # Define RNN
        self.rnn = nn.GRU(vocab_size, hidden_size, n_layers, batch_first=True)
        # Fully connected layer to output distribution of spam vs not spam
        self.decoder = nn.Linear(hidden_size, 2)
    
    def forward(self, inp, hidden=None):
        # Use one hot encodings for embedding tokens since the vocab size is small. Larger vocabs may require glove embeddings to get co-occurance information of a token in relation to the others.
        # Since we're building a model on characters, we can assume independence, so one hot encoding is appropriate. If words were token inputs, we would need to adjust for contextual vectors (GloVe)
        inp = self.ident[inp]
        # Get next output and hidden state
        output, hidden = self.rnn(inp, hidden)
        # Get prediction distribution over next tokens
        output = torch.cat([torch.max(output, dim=1)[0], torch.mean(output, dim=1)], dim=1)
        output = self.decoder(output)
        return output, hidden

## Part 3. Training [16 pt]

### Part (a) [4 pt]

Complete the `get_accuracy` function, which will compute the
accuracy (rate) of your model across a dataset (e.g. validation set).
You may modify `torchtext.data.BucketIterator` to make your computation
faster.

In [ ]:
def get_accuracy(model, data):
    """ Compute the accuracy of the `model` across a dataset `data`
    
    Example usage:
    
    >>> model = MyRNN() # to be defined
    >>> get_accuracy(model, valid) # the variable `valid` is from above
    """

### Part (b) [4 pt]

Train your model. Plot the training curve of your final model. 
Your training curve should have the training/validation loss and
accuracy plotted periodically.

Note: Not all of your batches will have the same batch size.
In particular, if your training set does not divide evenly by
your batch size, there will be a batch that is smaller than
the rest. 

### Part (c) [4 pt]

Choose at least 4 hyperparameters to tune. Explain how you tuned the hyperparameters.
You don't need to include your training curve for every model you trained.
Instead, explain what hyperparemters you tuned, what the best validation accuracy was,
and the reasoning behind the hyperparameter decisions you made.

For this assignment, you should tune more than just your learning rate and epoch. 
Choose at least 2 hyperparameters that are unrelated to the optimizer.

### Part (d) [2 pt]

Before we deploy a machine learning model, we usually want to have a better understanding
of how our model performs beyond its validation accuracy. An important metric to track is
*how well our model performs in certain subsets of the data*.

In particular, what is the model's error rate amongst data with negative labels?
This is called the **false positive rate**.

What about the model's error rate amongst data with positive labels?
This is called the **false negative rate**.

Report your final model's false positive and false negative rate across the
validation set.

In [ ]:
# Create a Dataset of only spam validation examples
valid_spam = torchtext.legacy.data.Dataset(
    [e for e in valid.examples if e.label == 1],
    valid.fields)
# Create a Dataset of only non-spam validation examples
valid_nospam = None # TODO

### Part (e) [2 pt]

The impact of a false positive vs a false negative can be drastically different.
If our spam detection algorithm was deployed on your phone, what is the impact
of a false positive on the phone's user? What is the impact of a false negative?

## Part 4. Evaluation [11 pt]

### Part (a) [1 pt]

Report the final test accuracy of your model.

### Part (b) [3 pt]

Report the false positive rate and false negative rate of your model across the test set.

### Part (c) [3 pt]

What is your model's prediction of the **probability** that
the SMS message "machine learning is sooo cool!" is spam?

Hint: To begin, use `text_field.vocab.stoi` to look up the index
of each character in the vocabulary.

In [ ]:
msg = "machine learning is sooo cool!"

### Part (d) [4 pt]

Do you think detecting spam is an easy or difficult task?

Since machine learning models are expensive to train and deploy, it is very
important to compare our models against baseline models: a simple
model that is easy to build and inexpensive to run that we can compare our
recurrent neural network model against.

Explain how you might build a simple baseline model. This baseline model
can be a simple neural network (with very few weights), a hand-written algorithm,
or any other strategy that is easy to build and test.

**Do not actually build a baseline model. Instead, provide instructions on
how to build it.**